In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [4]:
%load_ext autoreload
%autoreload 2
from scripts.text.article_text_processor import ArticleTextProcessor
from scripts.text.basic_text_processor import BasicTextProcessor
from scripts.text.teams_players import TeamPlayers
from scripts.text.extractive_summary.key_events_summary import KeyEventsSummary

from scripts.conf import TEAMS

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Eventos importantes

El objetivo es realizar un resumen extractivo basandose en eventos importantes de un partido de fútbol. 
La idea es:

- Obtener eventos importantes: resultado, goles y ocasiones por ejemplo
- Buscar en la noticia frases relacionadas con esos eventos, juntarlas y formar el resumen

In [5]:
key_events = ['goal']

In [6]:
processor = ArticleTextProcessor()
key_events = KeyEventsSummary(key_events)

In [7]:
all_files = processor.load_json()

In [8]:
season_file = 'premier_league_2019_2020.json'
league_season_teams = TEAMS[season_file.split('.')[0]]

In [9]:
prueba = all_files[season_file]['https://www.bbc.com/sport/football/49791610']

In [10]:
key_words = ['goal', 'red', 'attempt']

In [11]:
events = prueba['events']

In [12]:
processed_events = key_events.process_match_events(events, keep_key_events=True, league_season_teams=league_season_teams)

league_season_teams is empty. Initializing it to ['Liverpool', 'Manchester City', 'Leicester City', 'Chelsea', 'Manchester United', 'Wolverhampton Wanderers', 'Sheffield United', 'Tottenham Hotspur', 'Arsenal', 'Burnley', 'Crystal Palace', 'Everton', 'Newcastle United', 'Southampton', 'Brighton', 'West Ham', 'Watford', 'Bournemouth', 'Aston Villa', 'Norwich City']
Number of original events: 125
Number of processed events: 3


In [52]:
key_events.events_mapping_list

[28,
 29,
 34,
 35,
 44,
 45,
 47,
 48,
 50,
 54,
 59,
 62,
 63,
 64,
 65,
 68,
 71,
 72,
 77,
 78,
 82,
 97,
 110,
 113,
 116,
 117,
 118]

In [125]:
events[key_events.events_mapping_list[2]]

'Attempt saved. Nicolas Pépé (Arsenal) left footed shot from the centre of the box is saved in the bottom right corner. Assisted by Pierre-Emerick Aubameyang.'

In [126]:
processed_events[2]

'assisted shot saved Nicolas Pepe Pierre-Emerick Aubameyang corner box right arsenal footed attempt left'

__Buscando eventos en noticias__

Buscamos la frase más cercana a cada evento en cada artículo. Para ello, hacemos una aproximación
simple, usando un tfidf con el cuerpo de la noticia, y calculando la distancia coseno a cada
evento. 

Quizás este enfoque no funcione demasiado bien al ser las noticias tan cortas.

In [10]:
article = prueba['article']

In [49]:
article

'Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.\nIn a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nArsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang\'s cool finish. It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harry Maguire.\nGoalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford\'s late free-kick, while Bukayo Saka\'s goalbound shot crucially struck Victor Lindelof and flew over the top.\nMcTominay also headed a great chance well over from Ashley Young\'s corner but neither side could force a winner.\nThe result takes Arsenal into th

Necesitamos guardar todas las frases:

In [68]:
doc_sents_list = [str(sent) for sent in key_events.text_proc.get_sentences(article)]

In [69]:
len(doc_sents_list)

39

Procesamos las frases del artículo de la misma forma que los eventos:

In [16]:
processed_sentences = key_events.process_match_article(article)

In [132]:
len(processed_sentences)

39

In [153]:
processed_sentences

['old league stalemate Manchester United trafford challenge Arsenal away far clubs grim illustrate provided evidence',
 'attritional shadow disappointing lead Manchester United edge drive Scott McTominay gone gave half years encounter rising mighty end',
 "Pierre-Emerick Aubameyang's arsenal equalised cool finish hour courtesy",
 'offside ruled video confirmed onside assistant originally striker Harry Maguire',
 "excelled Bernd Leno goalbound goalkeeper shot Marcus Rashford's saves crucially kick Victor Lindelof Arsenal flew Bukayo Saka's fine struck",
 "chance McTominay corner force headed Ashley Young's winner",
 'worst games start failing finished means seven Manchester United time double arsenal season takes years',
 'captains led demonstrated circumstances instantly sides clubs',
 "old newly united led manchester divisive supporters Ashley Young Aaron Wan-Bissaka's Granit Xhaka service absence appointed captain Arsenal defensive injury",
 'decisions led fashion knack showing numer

Para usar el tfidf de sklearn, necesitamos proporcionar un iterable de elementos (frases en este caso):

In [17]:
processed_sentences_wc = ' '.join(processed_sentences)

También se tienen en cuenta bigramas para no perder las entidades nombradas (quizás habría que probar
con trigramas también):

In [110]:
vectorizer = CountVectorizer(ngram_range=(1,2), strip_accents='unicode')

In [112]:
X = vectorizer.fit_transform(processed_sentences).toarray()

In [113]:
vectorizer.get_feature_names()

['aaron',
 'aaron wan',
 'absence',
 'absence appointed',
 'action',
 'action decision',
 'action evasive',
 'appeal',
 'appearing',
 'appearing mctominay',
 'appointed',
 'appointed captain',
 'arsenal',
 'arsenal away',
 'arsenal defensive',
 'arsenal equalised',
 'arsenal felt',
 'arsenal flew',
 'arsenal point',
 'arsenal provide',
 'arsenal season',
 'arsenal unai',
 'ashley',
 'ashley young',
 'assistant',
 'assistant originally',
 'assisted',
 'assisted onside',
 'attritional',
 'attritional shadow',
 'aubameyang',
 'aubameyang arsenal',
 'aubameyang touch',
 'away',
 'away build',
 'away far',
 'ball',
 'ball box',
 'bernd',
 'bernd leno',
 'best',
 'best action',
 'best bournemouth',
 'better',
 'better chances',
 'bissaka',
 'bissaka granit',
 'blocks',
 'boss',
 'boss champions',
 'bournemouth',
 'bournemouth points',
 'box',
 'build',
 'build getting',
 'bukayo',
 'bukayo saka',
 'captain',
 'captain arsenal',
 'captains',
 'captains led',
 'catch',
 'catch manchester',
 'c

In [24]:
X.shape

(39, 520)

In [114]:
tfidfconverter = TfidfTransformer()

In [115]:
X = tfidfconverter.fit_transform(X).toarray()

In [116]:
X.shape

(39, 520)

Eventos

In [144]:
X_events = vectorizer.transform(processed_events)

In [145]:
X_events = tfidfconverter.transform(X_events).toarray()

In [30]:
X_events.shape

(27, 520)

In [91]:
X_events

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [147]:
count_ixs = np.argwhere(X_events > 0)

In [149]:
count_ixs

array([[  0,  52],
       [  0, 100],
       [  0, 270],
       [  0, 291],
       [  0, 295],
       [  0, 374],
       [  0, 414],
       [  0, 477],
       [  1,  12],
       [  1,  26],
       [  1,  52],
       [  1,  55],
       [  1,  56],
       [  1, 270],
       [  1, 343],
       [  1, 380],
       [  1, 414],
       [  2,  12],
       [  2,  26],
       [  2,  30],
       [  2,  52],
       [  2, 100],
       [  2, 139],
       [  2, 140],
       [  2, 270],
       [  2, 343],
       [  2, 351],
       [  2, 352],
       [  2, 374],
       [  2, 414],
       [  3,  52],
       [  3, 291],
       [  3, 295],
       [  3, 374],
       [  3, 414],
       [  3, 477],
       [  4,  26],
       [  4,  36],
       [  4,  52],
       [  4, 270],
       [  4, 291],
       [  4, 295],
       [  4, 296],
       [  4, 297],
       [  4, 370],
       [  4, 414],
       [  4, 477],
       [  5,  26],
       [  5,  52],
       [  5, 291],
       [  5, 295],
       [  5, 374],
       [  5,

In [ ]:
[  1,  12],
       [  1,  26],
       [  1,  52],
       [  1,  55],
       [  1,  56],
       [  1, 270],
       [  1, 343],
       [  1, 380],
       [  1, 414],

In [140]:
processed_events[1]

'outside wide shot high assisted Nicolas Pepe attempt box arsenal Bukayo Saka footed missed left'

In [150]:
pd.DataFrame(X_events, columns=vectorizer.get_feature_names()).iloc[1, [12,26,52,55,56,270,343,380,414]]

arsenal        0.207616
assisted       0.362105
box            0.362105
bukayo         0.325361
bukayo saka    0.362105
left           0.362105
pepe           0.325361
saka           0.299290
shot           0.362105
Name: 1, dtype: float64

In [152]:
doc_sents_list[4]

"Goalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.\n"

Distancia entre eventos y frases:

In [133]:
distances = cosine_similarity(X_events, X)

In [134]:
distances.shape

(27, 39)

In [151]:
distances[1]

array([0.02357063, 0.        , 0.0296338 , 0.        , 0.23169911,
       0.        , 0.02319782, 0.        , 0.01946064, 0.        ,
       0.        , 0.        , 0.        , 0.0412068 , 0.02066832,
       0.05721444, 0.        , 0.07172153, 0.        , 0.07333465,
       0.        , 0.17373423, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04316312, 0.        , 0.        ,
       0.        , 0.03889747, 0.        , 0.        , 0.11114276,
       0.        , 0.3253609 , 0.15101771, 0.        ])

In [136]:
distances.max(axis=1)

array([0.12384918, 0.3253609 , 0.3729923 , 0.14625438, 0.21650615,
       0.13250492, 0.3253609 , 0.20174099, 0.20161512, 0.42301245,
       0.14752496, 0.25041393, 0.38048124, 0.41625515, 0.34408246,
       0.17442482, 0.22877419, 0.14625438, 0.3473272 , 0.21650615,
       0.21457486, 0.13485292, 0.14752496, 0.24143053, 0.23138639,
       0.1796058 , 0.16062666])

In [137]:
sentences_ixs = distances.argmax(axis=1)

In [138]:
sentences_ixs

array([ 0, 36,  2,  0,  4,  0, 36, 13,  1,  2, 34,  1,  2,  2, 36, 21,  4,
        0,  5,  4,  3,  0, 34,  5,  4, 21, 21])

In [167]:
processed_events

['corner right left shot Manchester United saved box footed Andreas Pereira attempt',
 'high outside left shot arsenal Nicolas Pépé Bukayo Saka wide missed box footed assisted attempt',
 'corner right left shot arsenal Nicolas Pépé Pierre-Emerick Aubameyang saved box footed assisted attempt',
 'right outside shot blocked Paul Pogba Manchester United box footed attempt',
 'ball left shot blocked Paul Pogba Manchester United box footed assisted Marcus Rashford attempt',
 'right shot blocked Manchester United Jesse Lingard box footed assisted Andreas Pereira attempt',
 'corner right left shot arsenal Bukayo Saka saved box footed attempt',
 'corner right angle shot arsenal difficult saved footed attempt Matteo Guendouzi',
 'corner right outside left shot arsenal goal Manchester United Scott McTominay box footed assisted Marcus Rashford',
 'right shot lucas arsenal Pierre-Emerick Aubameyang misses missed box footed assisted attempt cross',
 'right outside blocked shot arsenal David Luiz box

In [58]:
doc_sents_list

[Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.,
 In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.,
 Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.,
 It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harry Maguire.,
 Goalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.,
 McTominay also headed a great chance well over from Ashley Young's corner but neither side could force a winner.,
 The result takes Arsenal into

In [139]:
for event_ix, sentence_ix in enumerate(sentences_ixs):
    print('Event:')
    print(events[key_events.events_mapping_list[event_ix]])
    print('Nearest sentence in article:')
    print(doc_sents_list[sentence_ix])

Event:
Attempt saved. Andreas Pereira (Manchester United) left footed shot from the centre of the box is saved in the bottom right corner.
Nearest sentence in article:
Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.

Event:
Attempt missed. Nicolas Pépé (Arsenal) left footed shot from outside the box is high and wide to the left. Assisted by Bukayo Saka.
Nearest sentence in article:
Bukayo
Event:
Attempt saved. Nicolas Pépé (Arsenal) left footed shot from the centre of the box is saved in the bottom right corner. Assisted by Pierre-Emerick Aubameyang.
Nearest sentence in article:
Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.
Event:
Attempt blocked. Paul Pogba (Manchester United) right footed shot from outside the box is blocked.
Nearest sentence in article:
Manchester United and Arsenal played out a grim

In [59]:
selected_article_sentences = sorted(list(set(sentences_ixs)))

In [60]:
selected_article_sentences

[0, 1, 2, 3, 4, 5, 13, 21, 34, 36]

In [ ]:
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [13]:
# Solo con goles
key_events.match_summary(prueba, count_vec_kwargs, save_relations=True)

Number of original events: 125
Number of processed events: 3
Number of sentences in original article: 37
Number of sentences in summary: 2


("In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nArsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.",
 array([1, 2, 2]),
 ['Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.\n',
  'In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\n',
  "Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.",
  'It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harr

In [166]:
key_events.match_summary(prueba, save_relations=True, ngram_range=(1,2), strip_accents='unicode')

Number of original events: 125
Number of processed events: 27
Number of sentences in original article: 39
Number of sentences in summary: 10
Event:
Attempt saved. Andreas Pereira (Manchester United) left footed shot from the centre of the box is saved in the bottom right corner.
Nearest sentence in article:
Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.

Event:
Attempt missed. Nicolas Pépé (Arsenal) left footed shot from outside the box is high and wide to the left. Assisted by Bukayo Saka.
Nearest sentence in article:
Bukayo
Event:
Attempt saved. Nicolas Pépé (Arsenal) left footed shot from the centre of the box is saved in the bottom right corner. Assisted by Pierre-Emerick Aubameyang.
Nearest sentence in article:
Arsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.
Event:
Attempt blocked. Paul Pogba (Manch

"Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.\nIn a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nArsenal equalised just before the hour courtesy of Pierre-Emerick Aubameyang's cool finish.It was originally ruled out for offside but the video assistant referee confirmed the striker had been played onside by Harry Maguire.\nGoalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.\nMcTominay also headed a great chance well over from Ashley Young's corner but neither side could force a winner.\nUnited had the edge on chances, altho

__Con jugadores más mencionados__

In [14]:
processed_events = key_events.process_match_events(events, keep_key_events=True,
                                                  league_season_teams=league_season_teams,
                                                  keep_top_players=2)

Adding events for ['Bukayo Saka', 'Matteo Guendouzi']
Added 21 new events for 2 player(s)
Number of original events: 125
Number of processed events: 24


In [15]:
key_events.match_summary(prueba, print_relations=True,
                         keep_top_players=2, ngram_range=(1,2), strip_accents='unicode')

Adding events for ['Bukayo Saka', 'Matteo Guendouzi']
Added 21 new events for 2 player(s)
Number of original events: 125
Number of processed events: 24
Number of sentences in original article: 37
Number of sentences in summary: 4
Event:
Foul by Matteo Guendouzi (Arsenal).
Nearest sentence in article:
United had the edge on chances, although they were grateful to David de Gea for a fine double save from Saka and Matteo Guendouzi in the first half
Event:
Hand ball by Bukayo Saka (Arsenal).
Nearest sentence in article:
Bukayo [Saka] is progressing well
Event:
Offside, Arsenal. Bernd Leno tries a through ball, but Bukayo Saka is caught offside.
Nearest sentence in article:
Bukayo [Saka] is progressing well
Event:
Bukayo Saka (Arsenal) wins a free kick in the attacking half.
Nearest sentence in article:
Bukayo [Saka] is progressing well
Event:
Matteo Guendouzi (Arsenal) wins a free kick in the defensive half.
Nearest sentence in article:
United had the edge on chances, although they were gr

('In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.\nUnited had the edge on chances, although they were grateful to David de Gea for a fine double save from Saka and Matteo Guendouzi in the first halfIt was left to the lethal Aubameyang to provide the finishing touch for Arsenal, correctly assisted by VAR after technology showed he was clearly onside. \nBukayo [Saka] is progressing well',
 array([13, 34, 34, 34, 13, 34, 34, 34, 13, 34, 13, 34, 13,  1, 34, 13, 34,
        21, 34, 13, 13, 13, 13, 34]))

## Para todos los partidos

In [4]:
key_events = KeyEventsSummary()

Casi 2h...

In [213]:
%%time
pd_df = key_events.run()

Number of original events: 108
Number of processed events: 40
Number of sentences in original article: 12
Number of sentences in summary: 6
Number of original events: 106
Number of processed events: 26
Number of sentences in original article: 6
Number of sentences in summary: 4
Number of original events: 105
Number of processed events: 26
Number of sentences in original article: 6
Number of sentences in summary: 4
Number of original events: 121
Number of processed events: 35
Number of sentences in original article: 5
Number of sentences in summary: 3
Number of original events: 116
Number of processed events: 30
Number of sentences in original article: 5
Number of sentences in summary: 5
Number of original events: 118
Number of processed events: 37
Number of sentences in original article: 5
Number of sentences in summary: 4
Number of original events: 117
Number of processed events: 28
Number of sentences in original article: 11
Number of sentences in summary: 8
Number of original events

/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 103
Number of processed events: 25
Number of sentences in original article: 12
Number of sentences in summary: 4
Number of original events: 100
Number of processed events: 33
Number of sentences in original article: 8
Number of sentences in summary: 3
Number of original events: 114
Number of processed events: 21
Number of sentences in original article: 6
Number of sentences in summary: 3
Number of original events: 112
Number of processed events: 40
Number of sentences in original article: 7
Number of sentences in summary: 3
Number of original events: 125
Number of processed events: 28
Could not perform summary for https://www.bbc.com/sport/football/50693021


/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 122
Number of processed events: 23
Could not perform summary for https://www.bbc.com/sport/football/50693028
Number of original events: 111
Number of processed events: 26
Number of sentences in original article: 6
Number of sentences in summary: 5
Number of original events: 91
Number of processed events: 15
Number of sentences in original article: 8
Number of sentences in summary: 3
Number of original events: 119
Number of processed events: 38
Number of sentences in original article: 12
Number of sentences in summary: 7
Number of original events: 118
Number of processed events: 31
Number of sentences in original article: 7
Number of sentences in summary: 4
Number of original events: 130
Number of processed events: 22
Number of sentences in original article: 16
Number of sentences in summary: 1
Number of original events: 108
Number of processed events: 30
Number of sentences in original article: 17
Number of sentences in summary: 4
Number of original events: 1

...

Number of original events: 116
Number of processed events: 17
Could not perform summary for https://www.bbc.com/sport/football/50537859
Number of original events: 107
Number of processed events: 24
Could not perform summary for https://www.bbc.com/sport/football/50537866
Number of original events: 106
Number of processed events: 26
Could not perform summary for https://www.bbc.com/sport/football/50530546
Number of original events: 80
Number of processed events: 21
Could not perform summary for https://www.bbc.com/sport/football/50530553
Number of original events: 92
Number of processed events: 30
Could not perform summary for https://www.bbc.com/sport/football/50530560
Number of original events: 87
Number of processed events: 22
Could not perform summary for https://www.bbc.com/sport/football/50530567
Number of original events: 117
Number of processed events: 26
Could not perform summary for https://www.bbc.com/sport/football/50530574
Number of original events: 109
Number of processed 

/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 100
Number of processed events: 24
Could not perform summary for https://www.bbc.com/sport/football/50281875
Number of original events: 96
Number of processed events: 23
Could not perform summary for https://www.bbc.com/sport/football/50281882
Number of original events: 103
Number of processed events: 18
Could not perform summary for https://www.bbc.com/sport/football/50274991
Number of original events: 95
Number of processed events: 20
Could not perform summary for https://www.bbc.com/sport/football/50274998
Number of original events: 108
Number of processed events: 17
Could not perform summary for https://www.bbc.com/sport/football/50275005
Number of original events: 106
Number of processed events: 23
Could not perform summary for https://www.bbc.com/sport/football/50275012
Number of original events: 91
Number of processed events: 32
Could not perform summary for https://www.bbc.com/sport/football/50275026
Number of original events: 99
Number of processed e

/home/carlos/PycharmProjects/tfm/scripts/text/key_events_summary.py:127: UserWarning: Could not perform tfidf
  warnings.warn('Could not perform tfidf')


Number of original events: 101
Number of processed events: 15
Number of sentences in original article: 6
Number of sentences in summary: 4
Number of original events: 94
Number of processed events: 15
Number of sentences in original article: 9
Number of sentences in summary: 4
Number of original events: 87
Number of processed events: 24
Number of sentences in original article: 9
Number of sentences in summary: 5
Number of original events: 74
Number of processed events: 11
Number of sentences in original article: 11
Number of sentences in summary: 5
Number of original events: 90
Number of processed events: 23
Number of sentences in original article: 5
Number of sentences in summary: 4
Number of original events: 99
Number of processed events: 22
Number of sentences in original article: 9
Number of sentences in summary: 3
Number of original events: 78
Number of processed events: 18
Number of sentences in original article: 9
Number of sentences in summary: 5
Number of original events: 137
N

__Lectura__

In [6]:
pd_df_summaries = pd.read_csv(CSV_DATA_PATH + 'summaries/key_events_summaries_1.csv')
pd_df_articles = pd.read_csv(CSV_DATA_PATH + 'articles_events.csv')

In [16]:
pd_df_articles[pd_df_articles.url=='https://www.bbc.com/sport/football/49861472']['article'].values

array(['Crystal Palace snatched victory at West Ham as Jordan Ayew had an 87th-minute goal awarded by the video assistant referee at the London Stadium.\nThe strike was initially ruled offside by the linesman after Ayew stabbed in from Martin Kelly\'s header but the decision was overturned after a lengthy VAR review.\nSebastien Haller had given West Ham the lead when he swept home.\nBut Palace equalised through Patrick van Aanholt\'s penalty before Ayew\'s goal secured all three points.\nThe win moved Roy Hodgson\'s side into the top four of the Premier League while ending West Ham\'s six-game unbeaten run.\nThe addition of VAR in the Premier League has been a much debated subject this season and the London Stadium has now played host to some of its most controversial moments.\nIn West Ham\'s opening game of the campaign against Manchester City, VAR ruled out a goal for the visitors after Raheem Sterling\'s shoulder was adjudged offside, and this was a similarly close call.\nWhen Ayew 

In [17]:
pd_df_articles[pd_df_articles.url=='https://www.bbc.com/sport/football/49861472']['events'].values

array(["['Offside, West Ham United. Mark Noble tries a through ball, but Andriy Yarmolenko is caught offside.', 'Foul by Jordan Ayew (Crystal Palace).', 'Mark Noble (West Ham United) wins a free kick in the attacking half.', 'Offside, Crystal Palace. James McArthur tries a through ball, but Jeffrey Schlupp is caught offside.', 'Wilfried Zaha (Crystal Palace) wins a free kick in the defensive half.', 'Foul by Aaron Cresswell (West Ham United).', 'Foul by Issa Diop (West Ham United).', 'Jordan Ayew (Crystal Palace) wins a free kick on the right wing.', 'Attempt blocked. Jordan Ayew (Crystal Palace) right footed shot from the left side of the box is blocked. Assisted by Gary Cahill.', 'Corner,  Crystal Palace. Conceded by Aaron Cresswell.', 'Offside, West Ham United. Ryan Fredericks tries a through ball, but Andriy Yarmolenko is caught offside.', 'Corner,  West Ham United. Conceded by Patrick van Aanholt.', 'Attempt saved. Sébastien Haller (West Ham United) right footed shot from the cent

In [13]:
pd_df_summaries[pd_df_summaries.url=='https://www.bbc.com/sport/football/49861472']['summary'].values

array(['Crystal Palace snatched victory at West Ham as Jordan Ayew had an 87th-minute goal awarded by the video assistant referee at the London Stadium.\nSebastien Haller had given West Ham the lead when he swept home.\nBut Palace equalised through Patrick van Aanholt\'s penalty before Ayew\'s goal secured all three points.\nMuch of their good form has been down to an attack that will be the envy of many of their opponents in the Premier League, with Haller, Andriy Yarmolenko, Felipe Anderson and Manuel Lanzini offering a combination of pace, power and creativity.\nHaller - West Ham\'s record signing in the summer - fluffed a golden chance in the first half when he shot straight at Palace goalkeeper Vicente Guaita from only a couple of yards out while Anderson mis-hit a volley before seeing a header saved. \nCrystal Palace manager Roy Hodgson, speaking to BBC Match of the Day: "I thought we were a bit unlucky to fall behind.'],
      dtype=object)

In [6]:
pd_df_art_summ = pd_df_articles.merge(pd_df_summaries, on=['json_file', 'url'])

In [22]:
pd_df_art_summ

,json_file,url,article,events,league,season,summary,article_sentences_ix,events_mapping
0,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,['Attempt blocked. Mario Balotelli (Brescia) r...,italian_serie_a,2019_2020,Mario Balotelli scored his first Serie A goal ...,[ 0 0 0 1 6 6 2 2 6 0 3 2 6 6 1 ...,"[2, 4, 5, 6, 8, 11, 13, 14, 16, 17, 21, 28, 29..."
1,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49873763,Fiorentina condemned AC Milan to a third conse...,"['Foul by Nikola Milenkovic (Fiorentina).', 'R...",italian_serie_a,2019_2020,Erick Pulgar's penalty opened the scoring befo...,[2 5 2 5 2 1 1 2 1 1 3 5 1 2 3 5 1 3 2 5 2 1 1...,"[4, 5, 8, 10, 11, 13, 17, 20, 22, 23, 29, 30, ..."
2,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49865134,Cristiano Ronaldo returned from injury with a ...,"['Corner, SPAL. Conceded by Matthijs de Ligt....",italian_serie_a,2019_2020,Cristiano Ronaldo returned from injury with a ...,[1 0 0 3 1 1 0 1 0 0 3 2 1 0 0 1 2 0 0 1 0 0 1...,"[3, 6, 8, 9, 13, 14, 15, 17, 32, 37, 39, 41, 4..."
3,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49866723,Alexis Sanchez scored and was sent off on his ...,"['Offside, Inter Milan. Milan Skriniar tries a...",italian_serie_a,2019_2020,Alexis Sanchez scored and was sent off on his ...,[4 0 0 0 4 0 0 1 1 4 0 0 0 4 4 1 0 0 0 0 4 4 1...,"[5, 16, 22, 23, 24, 25, 26, 27, 30, 35, 44, 45..."
4,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49832309,On-loan Manchester United defender Chris Small...,['Attempt missed. Alejandro Gómez (Atalanta) r...,italian_serie_a,2019_2020,On-loan Manchester United defender Chris Small...,[4 3 4 4 2 3 3 0 3 4 3 4 2 3 3 4 1 4 1 4 1 3 3...,"[2, 15, 16, 17, 18, 22, 23, 25, 28, 31, 47, 55..."
...,...,...,...,...,...,...,...,...,...
4355,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,After lacking the final product for much of 11...,['Attempt missed. Ilsinho (Philadelphia Union)...,mls,2019_2020,After lacking the final product for much of 11...,[1 1 1 1 3 8 0 3 1 8 1 1 1 1 1 1 4 3 1 1 8 1 9...,"[2, 8, 9, 11, 13, 14, 15, 17, 18, 24, 31, 32, ..."
4356,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,Jamiro Monteiro scored his second MLS goal on ...,['Chris Mavinga (Toronto FC) wins a free kick ...,mls,2019_2020,Jamiro Monteiro scored his second MLS goal on ...,[1 3 3 0 3 0 3 3 0 1 1 3 0 3 3 0 3],"[5, 10, 13, 19, 37, 48, 51, 56, 59, 62, 63, 66..."
4357,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,Nelson Bonilla's goal in first-half stoppage t...,['Bryan Tamacas (El Salvador) wins a free kick...,mls,2019_2020,El Salvador take the lead!!!\n\r\nWith the las...,[ 8 8 7 7 5 6 6 16 16 15],"[4, 21, 24, 42, 60, 61, 62, 87, 88, 99]"
4358,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,Haiti came back from a two-goal halftime defic...,"['Foul by Doneil Henry (Canada).', 'Herve Bazi...",mls,2019_2020,Alphonso Davies quick-kicked the ball to Jonat...,[6 2 2 2 6 3 7 6 7 2 6 7 6 2 2 2 2 3 3 2 2 2],"[4, 14, 19, 20, 22, 27, 32, 35, 37, 39, 47, 50..."


In [16]:
len(pd_df_art_summ)

4360

In [24]:
pd_df_art_summ[pd_df_art_summ.json_file=='premier_league_2019_2020.json']

,json_file,url,article,events,league,season,summary,article_sentences_ix,events_mapping
2746,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49791610,Manchester United and Arsenal played out a gri...,"['Foul by Matteo Guendouzi (Arsenal).', 'Scott...",premier_league,2019_2020,Manchester United and Arsenal played out a gri...,[ 0 36 2 0 4 0 36 13 1 2 34 1 2 2 36 ...,"[28, 29, 34, 35, 44, 45, 47, 48, 50, 54, 59, 6..."
2747,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49785376,"Brendan Rodgers says it will be a ""big ask"" fo...",['Attempt saved. Ayoze Pérez (Leicester City) ...,premier_league,2019_2020,Newcastle are anchored in the relegation zone ...,[30 37 30 30 30 6 24 24 30 30 30 30 8 19 9 ...,"[2, 9, 10, 12, 13, 23, 25, 28, 33, 38, 43, 48,..."
2748,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49778507,"Jurgen Klopp said Liverpool ""deserved"" victory...",['Attempt saved. Oliver McBurnie (Sheffield Un...,premier_league,2019_2020,"Jurgen Klopp said Liverpool ""deserved"" victory...",[59 0 43 59 29 21 20 21 59 0 3 59 26 59 59 ...,"[2, 6, 8, 11, 12, 13, 16, 17, 18, 19, 29, 34, ..."
2749,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49778502,Aston Villa boss Dean Smith says his side do n...,['Ben Mee (Burnley) wins a free kick in the de...,premier_league,2019_2020,Aston Villa boss Dean Smith says his side do n...,[20 2 43 1 2 0 28 1 8 2 1 2 1 2 29 ...,"[6, 9, 10, 11, 24, 25, 33, 34, 37, 38, 41, 42,..."
2750,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49778509,Left-back Aaron Cresswell fired in a late equa...,"['Corner, West Ham United. Conceded by Diego ...",premier_league,2019_2020,Left-back Aaron Cresswell fired in a late equa...,[ 0 23 23 2 37 11 0 34 11 23 23 37 23 23 53 ...,"[3, 8, 9, 13, 14, 24, 29, 30, 32, 36, 37, 44, ..."
...,...,...,...,...,...,...,...,...,...
2979,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49861475,Jeff Hendrick's splendid volley ensured Burnle...,['James Tarkowski (Burnley) wins a free kick i...,premier_league,2019_2020,Jeff Hendrick's splendid volley ensured Burnle...,[ 2 17 17 15 2 2 4 15 15 15 2 2 15 2 7 ...,"[7, 9, 10, 11, 14, 21, 25, 26, 33, 37, 38, 45,..."
2980,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49861470,James Milner's injury-time penalty earned a dr...,"['Offside, Leicester City. Wilfred Ndidi tries...",premier_league,2019_2020,James Milner's injury-time penalty earned a dr...,[32 0 0 12 0 26 32 32 32 32 32 12 32 26 15 ...,"[5, 10, 24, 29, 38, 39, 42, 48, 49, 60, 61, 68..."
2981,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49861473,Aston Villa's club record signing Wesley showe...,['Attempt saved. Anwar El Ghazi (Aston Villa) ...,premier_league,2019_2020,Aston Villa's club record signing Wesley showe...,[ 9 9 9 0 0 0 9 9 15 0 0 0 9 9 9 ...,"[2, 3, 6, 7, 8, 9, 10, 11, 14, 20, 21, 23, 24,..."
2982,premier_league_2019_2020.json,https://www.bbc.com/sport/football/49861474,Watford manager Quique Sanchez Flores said his...,"['Corner, Sheffield United. Conceded by José ...",premier_league,2019_2020,The hosts created the better chances at Vicara...,[ 8 20 33 33 25 17 17 25 2 25 25 5 25 33 25 ...,"[3, 5, 11, 13, 14, 25, 27, 28, 33, 37, 40, 49,..."


In [27]:
pd_df_art_summ[pd_df_art_summ.url=='https://www.bbc.com/sport/football/49778507']['article'].values

array(['Jurgen Klopp said Liverpool "deserved" victory at Sheffield United despite needing a "lucky" Georginio Wijnaldum goal to continue their perfect start to the Premier League season at a raucous Bramall Lane.\nThe winner was a moment to forget for the otherwise faultless Blades goalkeeper Dean Henderson, who spilled the Dutchman\'s tame effort before it squirmed agonisingly over the line.\n"It\'s normal you have to work hard for the points," said Klopp. "You have to stay concentrated and do the right things. We did today. It was never a clicking day, it was an average day.\n"We were deserved winners. If it\'s 0-0 we cannot moan and would not. But if there\'s one winner it should be us. We worked had for it. Winning in days like this is extremely worthy. We scored a lucky goal, we know that. But the boys worked so hard for it."\nKlopp\'s side were far from their best in South Yorkshire - and left thankful that the hosts could not find a way through in a game where they matched the 

In [29]:
pd_df_art_summ[pd_df_art_summ.url=='https://www.bbc.com/sport/football/49778507']['summary'].values

array(['Jurgen Klopp said Liverpool "deserved" victory at Sheffield United despite needing a "lucky" Georginio Wijnaldum goal to continue their perfect start to the Premier League season at a raucous Bramall Lane.\n"You have to stay concentrated and do the right things.Substitute Leon Clarke, making his Premier League debut at the age of 34, should have levelled it late on, but blazed over.\nLiverpool\'s first clear chance came as a result of a long ball over the top from Virgil van Dijk, but Sadio Mane\'s finish when through on goal was wild and wayward, while the Senegal forward later struck the post.\nAfter the break, United may have hit the front, but for alert defending from Joel Matip and Andrew Robertson to frustrate the enthusiastic John Lundstram.\nWith time running out, the Merseysiders withdrew Jordan Henderson for Divock Origi, and the move to a 4-2-3-1 formation ultimately paid off when Wijnaldum\'s shot snuck in.\nIndeed, they were lucky not to be punished when Matip and 

In [39]:
pd_df_art_summ[pd_df_art_summ.url=='https://www.bbc.com/sport/football/49778507']['article_sentences_ix']

2748    [59  0 43 59 29 21 20 21 59  0  3 59 26 59 59 ...
Name: article_sentences_ix, dtype: object